In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import time

from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interact
import ipywidgets as widgets

In [2]:
def phi(x, y, L):
    """Initial function. In particular, it also defines the edge conditions."""
    if x == 0 or x == L:
        return 1
    return 0

In [3]:
def t_iterations(x, y, L, t):
    """Simulate the random walk during n iterations, starting from point (x, y).
    Return the product of the values taken by all branches during the walk from time 0 to t in a list of length t+1.
    """
    alpha = 1/L**2  # probability of dying = probability of duplicating
    values = [phi(x, y, L)] + [0]*t
    aliveBranches = np.zeros((L+1, L+1))  # the number of alive processes at position 0 <= x,y <= L
    aliveBranches = [[x,y]]  # initially we have one branch
    for n in range(1, t+1):
        if aliveBranches == []:
            # all branches have died. The process will no longer evolve,
            # and we can keep the values initialized to 0 for all future times
            return values
        
        branchesToDelete = []  # indices of the branches that will die during this iteration
        branchesToAdd = []  # coordinates of the branches that be created (by duplication) during this iteration
        for branchId, (x, y) in enumerate(aliveBranches):
            if not (x == 0 or x == L or y == 0 or y == L):
                # we are neither on the border nor in the cemetery
                rand = random.random()
                if 0 <= rand < 1 - 2*alpha:
                    # Move the branch left, right, up or down, with equal probabilities.
                    if 0 <= rand < 1/4*(1 - 2*alpha):
                        x -= 1  # go left
                    elif 1/4*(1 - 2*alpha) <= rand < 2/4*(1 - 2*alpha):
                        x += 1  # go right
                    elif 2/4*(1 - 2*alpha) <= rand < 3/4*(1 - 2*alpha):
                        y -= 1  # go down
                    else:
                        y += 1  # go up
                    aliveBranches[branchId] = [x, y]

                elif 1 - 2*alpha <= rand < 1 - alpha:
                    # This branch dies with probability alpha
                    branchesToDelete.append(branchId)
                else:
                    # Duplicate this branch with probability alpha
                    branchesToAdd.append([x,y])

        # Remove the dead branches
        aliveBranches = [branch for i, branch in enumerate(aliveBranches) if i not in branchesToDelete]
        # Add the newly created branches
        aliveBranches += branchesToAdd
        
        # Compute the product of the values of phi for each of our alive branches
        if aliveBranches != []:
            values[n] = 1  # initialize product
            for x, y in aliveBranches:
                values[n] *= phi(x, y, L)
    return values

In [4]:
def monte_carlo(x, y, L, K, t):    
    """Simulate K times the evolution up to time t starting from point
    (x, y), and return the average result in a list of length t."""
    temporary_sum = [0]*(t+1)
    for i in range(K):
        sample = t_iterations(x, y, L, t)  # list of length t + 1
        temporary_sum = [current + new for current, new in zip(temporary_sum, sample)]
    return [x/K for x in temporary_sum]

In [5]:
def approximate_solution(L, K, t):
    """Approximate solution at time t.
    Return a list of length t, containing the approximate solution (array of size (L, L) for all intermediate times)"""
    averages = [np.zeros((L, L)) for _ in range(t+1)]
    start = time.time()
    for x in range(L):
        for y in range(L):
            # The square associated with (x, y) is the one whose bottom left
            # corner is at (x, y)
            this_pos_monte_carlo = monte_carlo(x, y, L, K, t) # list of values until from 0 to t
            for n in range(t+1):
                averages[n][x][y] = this_pos_monte_carlo[n]
        nowm = int((time.time() - start) // 60)
        nows = int((time.time() - start) % 60)
        print("x = {:2}/{} done, elapsed time = {:2}m {:2}s".format(x, L-1, nowm, nows))
    return averages

In [6]:
def display(L, K, t):
    approx_intermediate_images = approximate_solution(L, K, t)
    def _show(frame=widgets.IntSlider(min=0,max=t-1,step=1,value=0)):
        fig, ax = plt.subplots(figsize=(7,7))
        ax.imshow(approx_intermediate_images[frame][:][:].T, origin="lower",
                   extent=[0, 1, 0, 1], cmap="jet")
    interact(_show)

In [7]:
def test():
    """Main function."""
    L = 40  # discretize with squares of length 1/L
    K = 100  # number of simulations per point to compute the average result
    t = 200
    print("L = {}, K = {}, t = {}".format(L, K, t))
    display(L, K, t)

In [8]:
test()

L = 40, K = 100, t = 200
x =  0/39 done, elapsed time =  0m  1s
x =  1/39 done, elapsed time =  0m  2s
x =  2/39 done, elapsed time =  0m  4s
x =  3/39 done, elapsed time =  0m  6s
x =  4/39 done, elapsed time =  0m  8s
x =  5/39 done, elapsed time =  0m 10s
x =  6/39 done, elapsed time =  0m 12s
x =  7/39 done, elapsed time =  0m 14s
x =  8/39 done, elapsed time =  0m 16s
x =  9/39 done, elapsed time =  0m 18s
x = 10/39 done, elapsed time =  0m 21s
x = 11/39 done, elapsed time =  0m 23s
x = 12/39 done, elapsed time =  0m 25s
x = 13/39 done, elapsed time =  0m 27s
x = 14/39 done, elapsed time =  0m 29s
x = 15/39 done, elapsed time =  0m 32s
x = 16/39 done, elapsed time =  0m 34s
x = 17/39 done, elapsed time =  0m 36s
x = 18/39 done, elapsed time =  0m 38s
x = 19/39 done, elapsed time =  0m 40s
x = 20/39 done, elapsed time =  0m 43s
x = 21/39 done, elapsed time =  0m 45s
x = 22/39 done, elapsed time =  0m 47s
x = 23/39 done, elapsed time =  0m 50s
x = 24/39 done, elapsed time =  0m 52s


interactive(children=(IntSlider(value=0, description='frame', max=199), Output()), _dom_classes=('widget-inter…